# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [59]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [60]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.561470,-52.712616,14.66,91,100,3.09,CA,2024-09-30
1,1,kapuskasing,49.416667,-82.433333,20.00,40,20,5.66,CA,2024-09-30
2,2,iqaluit,63.749440,-68.521857,7.97,70,75,2.57,CA,2024-09-30
3,3,afif,23.902370,42.915085,27.77,15,2,4.62,SA,2024-09-30
4,4,sete,43.401443,3.695977,18.92,79,48,2.83,FR,2024-09-30


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [61]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
	"Lng",
	"Lat",
    geo=True,
    frame_width = 600,
    frame_height = 500,
	tiles="OSM",
	size="Humidity",
    color="City",
    alpha=0.5
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [62]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()
ideal_city_data_df = ideal_city_data_df.reset_index()

# Display sample data
ideal_city_data_df

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,32,32,forest grove,45.519745,-123.111441,21.22,41,0,2.68,US,2024-09-30
1,84,84,'aqrah,36.730486,43.874893,23.29,23,0,1.75,IQ,2024-09-30
2,90,90,maintirano,-18.060895,44.023951,24.80,78,0,1.50,MG,2024-09-30
3,142,142,college,45.326647,-75.838557,23.95,56,0,4.02,CA,2024-09-30
4,153,153,mohr,27.551465,52.882125,26.15,73,0,1.43,IR,2024-09-30
5,186,186,la foa,-21.709694,165.825930,22.54,77,0,0.90,FR,2024-09-30
6,187,187,bizerte,37.272090,9.870857,21.99,60,0,4.12,TN,2024-09-30
7,192,192,al jawf,24.198549,23.293897,26.91,36,0,4.03,LY,2024-09-30
8,249,249,at taj,24.217316,23.298738,26.91,36,0,4.03,LY,2024-09-30
9,268,268,camalu,30.842355,-116.062745,22.91,62,0,3.87,MX,2024-09-30


### Step 3: Create a new DataFrame called `hotel_df`.

In [63]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,forest grove,US,45.519745,-123.111441,41,
1,'aqrah,IQ,36.730486,43.874893,23,
2,maintirano,MG,-18.060895,44.023951,78,
3,college,CA,45.326647,-75.838557,56,
4,mohr,IR,27.551465,52.882125,73,
5,la foa,FR,-21.709694,165.825930,77,
6,bizerte,TN,37.272090,9.870857,60,
7,al jawf,LY,24.198549,23.293897,36,
8,at taj,LY,24.217316,23.298738,36,
9,camalu,MX,30.842355,-116.062745,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [64]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
    "filter" : "",
    "bias" : ""
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(
        url = base_url,
        params = params
    )

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
forest grove - nearest hotel: Best Western
'aqrah - nearest hotel: ھۆتێل سیپەل
maintirano - nearest hotel: Hotel Safari
college - nearest hotel: Hyatt Place Ottawa West
mohr - nearest hotel: هتل پارس
la foa - nearest hotel: Oasis de Tendea
bizerte - nearest hotel: Hôtel Zitouna
al jawf - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
camalu - nearest hotel: Margaritaville
san isidro - nearest hotel: No hotel found
sanshui - nearest hotel: Lucky Garden
sur - nearest hotel: El Boutique Hotel
papao - nearest hotel: No hotel found
arar - nearest hotel: دار أوتيل Dar Otel
quetta - nearest hotel: Al Abbas Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
0,forest grove,US,45.519745,-123.111441,41,Best Western
1,'aqrah,IQ,36.730486,43.874893,23,ھۆتێل سیپەل
2,maintirano,MG,-18.060895,44.023951,78,Hotel Safari
3,college,CA,45.326647,-75.838557,56,Hyatt Place Ottawa West
4,mohr,IR,27.551465,52.882125,73,هتل پارس
5,la foa,FR,-21.709694,165.825930,77,Oasis de Tendea
6,bizerte,TN,37.272090,9.870857,60,Hôtel Zitouna
7,al jawf,LY,24.198549,23.293897,36,No hotel found
8,at taj,LY,24.217316,23.298738,36,No hotel found
9,camalu,MX,30.842355,-116.062745,62,Margaritaville


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
%%capture --no-display

# Configure the map plot
refined_map_plot = hotel_df.hvplot.points(
	"Lng",
	"Lat",
    geo=True,
    frame_width = 600,
    frame_height = 500,
	tiles="OSM",
	size="Humidity",
    color="City",
    alpha=0.5,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
refined_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)